In [2]:
# Import findspark and initialise. 
import findspark
findspark.init()

In [1]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/18 13:20:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=',', header=True, inferSchema=True)

df.show(10)


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [6]:
print(df.columns)

['id', 'date', 'date_built', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view']


In [5]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView('home')


In [12]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql(
"""
select date_built, bedrooms, round(avg(price), 2) as Average_price from home where bedrooms = 4
group by date_built, bedrooms

"""
).show()


+----------+--------+-------------+
|date_built|bedrooms|Average_price|
+----------+--------+-------------+
|      2014|       4|    299073.89|
|      2012|       4|    298233.42|
|      2015|       4|    307908.86|
|      2011|       4|     302141.9|
|      2013|       4|    299999.39|
|      2016|       4|    296050.24|
|      2017|       4|    296576.69|
|      2010|       4|    296800.75|
+----------+--------+-------------+



In [17]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

spark.sql(
    """
    select date_built, bedrooms, bathrooms, round(avg(price), 2) as Average_price 
    from home where bedrooms = 3 and bathrooms = 3
    group by date_built, bedrooms, bathrooms
    order by date_built
"""
).show()



+----------+--------+---------+-------------+
|date_built|bedrooms|bathrooms|Average_price|
+----------+--------+---------+-------------+
|      2010|       3|        3|    292859.62|
|      2011|       3|        3|    291117.47|
|      2012|       3|        3|    293683.19|
|      2013|       3|        3|    295962.27|
|      2014|       3|        3|    290852.27|
|      2015|       3|        3|     288770.3|
|      2016|       3|        3|    290555.07|
|      2017|       3|        3|    292676.79|
+----------+--------+---------+-------------+



In [21]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

spark.sql(
    """
    select date_built, bedrooms, bathrooms, floors, sqft_living, round(avg(price), 2)
    from home where bedrooms = 3 and bathrooms = 3 and floors = 2 and sqft_living >= 2000
    group by date_built, bedrooms, bathrooms, floors, sqft_living
    order by date_built
"""
).show()


+----------+--------+---------+------+-----------+--------------------+
|date_built|bedrooms|bathrooms|floors|sqft_living|round(avg(price), 2)|
+----------+--------+---------+------+-----------+--------------------+
|      2010|       3|        3|     2|       2131|            279680.0|
|      2010|       3|        3|     2|       2111|            259382.0|
|      2010|       3|        3|     2|       2172|            266280.0|
|      2010|       3|        3|     2|       2055|            177217.0|
|      2010|       3|        3|     2|       2395|            123326.0|
|      2010|       3|        3|     2|       2313|            130941.0|
|      2010|       3|        3|     2|       2149|            228717.0|
|      2010|       3|        3|     2|       2215|            145507.0|
|      2010|       3|        3|     2|       2387|            383197.0|
|      2010|       3|        3|     2|       3945|            890532.0|
|      2010|       3|        3|     2|       2483|            22

In [25]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql(
    """
    select round(avg(price), 2) as Average_price, view from home 
    group by view
    having avg(home.price) >= 350000
"""
).show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+----+
|Average_price|view|
+-------------+----+
|   1056336.74|  85|
|    736679.93|  65|
|     755214.8|  53|
|   1080649.37|  78|
|   1053472.79|  81|
|   1058802.78|  76|
|   1137372.73|  91|
|   1026006.06|  93|
|    733780.26|  52|
|   1070444.25|  86|
|    1033536.2|  94|
|     734340.5|  57|
|    798684.82|  54|
|   1017815.92|  96|
|    970402.55|  92|
|    767036.67|  64|
|    746877.59|  61|
|   1031719.35|  88|
|    780914.67|  72|
|     791453.0|  59|
+-------------+----+
only showing top 20 rows

--- 0.5212507247924805 seconds ---


In [26]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home")


DataFrame[]

In [28]:
# 8. Check if the table is cached.
spark.catalog.isCached('home')

True

In [29]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql(
    """
    select round(avg(price), 2) as Average_price, view from home 
    group by view
    having avg(home.price) >= 350000
"""
).show()

print("--- %s seconds ---" % (time.time() - start_time))


+-------------+----+
|Average_price|view|
+-------------+----+
|   1056336.74|  85|
|    736679.93|  65|
|     755214.8|  53|
|   1080649.37|  78|
|   1053472.79|  81|
|   1058802.78|  76|
|   1137372.73|  91|
|   1026006.06|  93|
|    733780.26|  52|
|   1070444.25|  86|
|    1033536.2|  94|
|     734340.5|  57|
|    798684.82|  54|
|   1017815.92|  96|
|    970402.55|  92|
|    767036.67|  64|
|    746877.59|  61|
|   1031719.35|  88|
|    780914.67|  72|
|     791453.0|  59|
+-------------+----+
only showing top 20 rows

--- 0.26627302169799805 seconds ---


In [30]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 

df.write.partitionBy("date_built").mode('overwrite').parquet("parquet_home")

In [31]:
# 11. Read the formatted parquet data.

parq_df = spark.read.parquet('parquet_home')
parq_df.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [32]:
# 12. Create a temporary table for the parquet data.
parq_df.createOrReplaceTempView('p_home')

In [33]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql(
    """
    select round(avg(price), 2) as Average_price, view from home 
    group by view
    having avg(home.price) >= 350000
"""
).show()


print("--- %s seconds ---" % (time.time() - start_time))

+-------------+----+
|Average_price|view|
+-------------+----+
|   1056336.74|  85|
|    736679.93|  65|
|     755214.8|  53|
|   1080649.37|  78|
|   1053472.79|  81|
|   1058802.78|  76|
|   1137372.73|  91|
|   1026006.06|  93|
|    733780.26|  52|
|   1070444.25|  86|
|    1033536.2|  94|
|     734340.5|  57|
|    798684.82|  54|
|   1017815.92|  96|
|    970402.55|  92|
|    767036.67|  64|
|    746877.59|  61|
|   1031719.35|  88|
|    780914.67|  72|
|     791453.0|  59|
+-------------+----+
only showing top 20 rows

--- 0.2574648857116699 seconds ---


In [34]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home")

DataFrame[]

In [35]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home")


False